[View in Colaboratory](https://colab.research.google.com/github/vivekster12/Parser/blob/master/Train_NER.ipynb)

In [0]:
from __future__ import unicode_literals, print_function
import random
import spacy
import ast
from pathlib import Path

In [0]:
!pip install spacy

In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import io
import ast

In [0]:
uploaded = files.upload()

In [0]:
corpus = pd.read_csv(io.StringIO(uploaded['Opioid_FREETEXT_HANDCODING.csv'].decode('CP1252')),sep=',')

In [0]:
prescriptions = corpus['freetext_posology'].values

In [128]:
up = files.upload()

Saving new_NER_reg.csv to new_NER_reg.csv


In [0]:
NER_reg = pd.read_csv(io.StringIO(up['new_NER_reg.csv'].decode('utf-8')),sep=',', header = None)

In [0]:
TRAIN_CORPUS = []
for sample in NER_reg.values:
#   print (type(sample[1]))
  print ((sample)[1])
  sample[1] = (ast.literal_eval(sample[1]))
  
  TRAIN_CORPUS.append(tuple(sample))

In [0]:
for text, annotations in TRAIN_CORPUS:
  for ent in annotations.get('entities'):
    if ('' in ent):
      print (text)
      print (ent)
#       annotations.get('entities').remove(ent)

In [0]:
print (len(TRAIN_CORPUS))
for line in TRAIN_CORPUS:
  print (line)

In [0]:
# entity labels 
L_FREQ = 'FREQ'
L_VAR_FREQ = 'VAR_FREQ'
L_DOSE = 'DOSE'
L_VAR_DOSE = 'VAR_DOSE'
L_PRN = 'PRN'
L_VAR_OVER_TIME = 'VAR_OVER_TIME'
L_STRENGTH = 'STRENGTH'
L_UNITS = 'UNITS'
L_NUM_TABS = 'NUM_TABS'
L_VAR_NUM_TABS = 'VAR_NUM_TABS'
L_TABS = 'TABS'

In [0]:
def spacy_ner(TRAIN_DATA, VALIDATION_DATA, n_iter, model=None, new_model_name='TRIAL',output_dir=None):
    out = []
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _,annotations in TRAIN_DATA:
#       print (type(annotations))
      for ent in annotations.get('entities'):
        ner.add_label(ent[2])
   

    # add new entity label to entity recognizer
    ner.add_label(L_DOSE)
    ner.add_label(L_VAR_DOSE)
    ner.add_label(L_FREQ)
    ner.add_label(L_VAR_FREQ)
    ner.add_label(L_PRN)
    ner.add_label(L_VAR_OVER_TIME)
    ner.add_label(L_STRENGTH)
    ner.add_label(L_UNITS)
    ner.add_label(L_NUM_TABS)
    ner.add_label(L_VAR_NUM_TABS)
    ner.add_label(L_TABS)
    
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()
    
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
#                 print (text, annotations)
                nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                           losses=losses)
            print(losses)


    # save model to output directory
    if output_dir is None:
        print ('here')
        output_dir = Path('Libraries\Documents')
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)
        
        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for i,(text,_) in enumerate(VALIDATION_DATA):
            NER = []
            NER.append(text)
            print (text, '***')
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#             print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
            NER.append([(ent.text, ent.label_) for ent in doc.ents])
            out.append(NER)
   
    return out

In [0]:
from sklearn.model_selection import KFold

In [0]:
def do_k_fold_cross(k,num_it):
  kf = KFold(n_splits=k, shuffle=True)
  kf.get_n_splits(TRAIN_CORPUS)
  print(kf)
  k_count = 1
  presc = []
  score = []
  pred_arr = []
  for i,j in kf.split(TRAIN_CORPUS, y=None):
  # i is train indices
  # j is test indices
    TRAIN_DATA = []
    VALIDATION_DATA = []
    for index in i:
      TRAIN_DATA.append(TRAIN_CORPUS[index])
    for index in j:
      VALIDATION_DATA.append(TRAIN_CORPUS[index])
      presc.append(TRAIN_CORPUS[index][0])

    parse_output = spacy_ner(TRAIN_DATA,VALIDATION_DATA,num_it)
    results, predictions = organize_output(parse_output, VALIDATION_DATA, pred_arr)
    score_table = gen_score(results, score)
    # next two lines come outside the loop
  get_acc(score_table,presc)
  return presc,pred_arr,score_table,k,num_it


In [136]:
presc,pred_arr,score_table,k,num_it= do_k_fold_cross(4,20)

KFold(n_splits=4, random_state=None, shuffle=True)
Created blank 'en' model
{'ner': 937.5292112537797}
{'ner': 402.13679794131355}
{'ner': 307.6228917064506}
{'ner': 256.060436838323}
{'ner': 268.1449636414752}
{'ner': 175.5457291494806}
{'ner': 180.28248918007694}
{'ner': 134.71234873623672}
{'ner': 136.37504686893794}
{'ner': 137.69192453049484}
{'ner': 126.47615808646219}
{'ner': 101.50057393520177}
{'ner': 83.02534047371398}
{'ner': 117.65790851972388}
{'ner': 113.43994191038524}
{'ner': 104.75448747976174}
{'ner': 78.11532082017723}
{'ner': 72.10989774971407}
{'ner': 99.45217151009521}
{'ner': 54.512908720092454}
here
Loading from Libraries\Documents
patch 5 mcg/hr Apply 1 patch TD q7days ***
Entities []
50 mg po bid prn, 20 tablets dispensed 23SEP2016 ***
Entities [('50', 'DOSE'), ('mg', 'QUANTITY'), ('bid', 'FREQ'), ('prn', 'PRN')]
1-2 mg po q4h PRN for 2 weeks ***
Entities [('1-2', 'VAR_DOSE'), ('mg', 'QUANTITY'), ('q4h', 'FREQ'), ('PRN', 'PRN')]
1 mg PO q4-6h PRN (total 15 tab

In [0]:
def organize_output(parse_output, VALIDATION_DATA, pred_arr):
  org_res = []
  entities = ['DOSE', 'VAR_DOSE', 'QUANTITY', 'FREQ', 'VAR_FREQ', 'PRN', 'VAR_OVER_TIME', 'STRENGTH', 'UNITS', 'NUM_TABS', 'VAR_NUM_TABS', 'TABS']
  
  for i,line in enumerate(parse_output): 
    table = dict((el,[]) for el in entities)
  
  #   format of dict: 'key': ['pred', 'actual']
    pred_arr.append(line[1])
#     print (line[0])
    for p_ent in line[1]:
      if (p_ent[1] in table):
        table[p_ent[1]].append(p_ent[0])
      else:
        print ('this should not happen')
    
#     print (VALIDATION_DATA[i][0])
    
    for a_ent in VALIDATION_DATA[i][1].get('entities'):
      if (a_ent[2] in table):
        table[a_ent[2]].append(VALIDATION_DATA[i][0][a_ent[0]:a_ent[1]])
      else:
        print ('this should not happen')

#     print ('here')
    org_res.append(table)
  return org_res, pred_arr

In [0]:
# scoring convention: None --> no actual value, no pred value, 1 --> pred value = actual value, 
# 0 --> pred value != actual value (either it predicted wrong value or did not predict anything when it should have)
def gen_score(results, score_arr):
  entities = ['DOSE', 'VAR_DOSE', 'QUANTITY', 'FREQ', 'VAR_FREQ', 'PRN', 'VAR_OVER_TIME', 'STRENGTH', 'UNITS', 'NUM_TABS', 'VAR_NUM_TABS', 'TABS']

  for i,line in enumerate(results):
    score = [np.nan] * len(entities)
    
    
    for key, value in line.items():
      if (len(value) == 0):
        continue

      elif (len(value)%2 == 0):
#         print ('enter')
        p1 = 0
        p2 = int(len(value)/2)
        wrong = 0
        while (True):
#           print (len(value))
#           print (p1)
#           print (p2)
          # check if values (pred and act) are equal
          if (p2 >= len(value) and wrong == 0):
            score[entities.index(key)] = 1
            break
          elif (value[p1] != value[p2]):
            score[entities.index(key)] = 0
            wrong = 1
            break
          else:
            p1 +=1
            p2 +=1
        
#         print ('exit')

      else:
        score[entities.index(key)]= 0

    score_arr.append(score)
#     print (score)
  
  return score_arr
  

In [0]:
def get_acc(score_table, presc):
  for line in score_table:
    if (0 not in line):
      line.append(1)
    else:
      line.append(np.nan)

  #   conv to np array
  matrix = np.array(score_table)

  temp = []
  presc.append('Accuracy Measures')
  for i,ent in enumerate(matrix.T):
    if (i != 12):
      acc = np.count_nonzero(ent == 1)/sum(~np.isnan(ent))
    else:
      acc = np.count_nonzero(ent == 1)/len(ent)
    temp.append(acc)
  
  score_table.append(temp)


In [0]:
def to_csv(presc,pred_arr,score_table,k,num_it):
  pred_arr.append(np.nan)
  csv = []
  for index,(_,_,_) in enumerate(zip(presc,pred_arr,score_table)):
    temp = []
    temp.append(presc[index])
    temp.append(pred_arr[index])
    temp.extend(score_table[index])
    print (len(temp))
    csv.append(temp)
    
  df = pd.DataFrame(data = csv)
  df.to_csv('k_fold_res_' + str(k) + '_' + str(num_it) + '.csv', index = False, header = ['prescription','pred_values','DOSE', 'VAR_DOSE', 'QUANTITY', 'FREQ', 'VAR_FREQ', 'PRN', 'VAR_OVER_TIME', 'STRENGTH', 'UNITS', 'NUM_TABS', 'VAR_NUM_TABS', 'TABS', 'perfect_match'])
  
  # Create & upload a file.
  uploaded = drive.CreateFile({'title': 'k_fold_res_' + str(k) + '_' + str(num_it) + '.csv'})
  uploaded.SetContentFile('k_fold_res_' + str(k) + '_' + str(num_it) + '.csv')
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
to_csv(presc,pred_arr,score_table,k,num_it)